**IMPORTS** and **EXTRACTING CSV DATA**

In [45]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

# replace the file 'Audiobooks_data.csv' with your csv file path and name
raw_csv_data = np.loadtxt( 'Audiobooks_data.csv', delimiter = ',')

# inputs exclude first and last column of csv file, modify according to your use
unscaled_inputs_all = raw_csv_data[:, 1:-1]

# targets are in the last column in csv file
target_all = raw_csv_data[:, -1]

**BALANCE THE DATASET**

In [46]:
# This is just to balance the target datset ,i.e. there should be equal number of rows for each target value
num_one_targets = int(np.sum(target_all))
zero_targets_counter = 0
indices_to_be_removed = []

for i in range(target_all.shape[0]):
    if target_all[i] == 0:
        zero_targets_counter+=1
        if zero_targets_counter > num_one_targets:
            indices_to_be_removed.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_be_removed, axis = 0)
target_equal_priors = np.delete(target_all, indices_to_be_removed, axis = 0)

**STANDARDIZE THE INPUT**

In [47]:
# preprocessing the input to scale all the values relatively
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)


**SHUFFLE THE DATA**

In [48]:
# random shuffle the data for better results and accuracy
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = target_equal_priors[shuffled_indices]

**SPLIT DATA INTO TRAIN, VALDIATION AND TEST**

In [49]:
# Portion of train, validation and test dataset
TRAIN_PORTION = 0.8
VALIDATION_PORTION = 0.1

samples_count = shuffled_inputs.shape[0]

train_samples_count = int(TRAIN_PORTION*samples_count)
validation_samples_count = int(VALIDATION_PORTION*samples_count)
test_samples_count = samples_count- train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [50]:
# Replace the 'Audiobook_data_' with whatever you want, this code will save npz files for model in your directory
np.savez('Audiobook_data_train', inputs= train_inputs, targets= train_targets)
np.savez('Audiobook_data_validation', inputs= validation_inputs, targets= validation_targets)
np.savez('Audiobook_data_test', inputs= test_inputs, targets= test_targets)

In [51]:
# Extracting data from saved .npz files
npz = np.load('Audiobook_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobook_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobook_data_test.npz')

test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

**MODEL**

In [52]:
# change input size to "number of varaibles that can affect your result"
# you can play around "hidden_layer_size"
input_size = 10
output_size = 2
hidden_layer_size =50

# training the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

# sparse_categorical_crossentropy is used for one-hot encoding
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# these values can also be changed accordingly, so play around
batch_size =100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, 
          train_targets,
          batch_size = batch_size,
          callbacks = [early_stopping],
          epochs = max_epochs,
          validation_data = (validation_inputs, validation_targets),
          verbose = 2
        )



Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 0s - loss: 0.5153 - accuracy: 0.7863 - val_loss: 0.4078 - val_accuracy: 0.8658
Epoch 2/100
3579/3579 - 0s - loss: 0.3512 - accuracy: 0.8790 - val_loss: 0.3443 - val_accuracy: 0.8725
Epoch 3/100
3579/3579 - 0s - loss: 0.3088 - accuracy: 0.8852 - val_loss: 0.3211 - val_accuracy: 0.8792
Epoch 4/100
3579/3579 - 0s - loss: 0.2914 - accuracy: 0.8924 - val_loss: 0.3039 - val_accuracy: 0.8814
Epoch 5/100
3579/3579 - 0s - loss: 0.2800 - accuracy: 0.8938 - val_loss: 0.2950 - val_accuracy: 0.8814
Epoch 6/100
3579/3579 - 0s - loss: 0.2717 - accuracy: 0.8969 - val_loss: 0.2867 - val_accuracy: 0.8881
Epoch 7/100
3579/3579 - 0s - loss: 0.2664 - accuracy: 0.9003 - val_loss: 0.2789 - val_accuracy: 0.8949
Epoch 8/100
3579/3579 - 0s - loss: 0.2609 - accuracy: 0.9025 - val_loss: 0.2754 - val_accuracy: 0.8993
Epoch 9/100
3579/3579 - 0s - loss: 0.2556 - accuracy: 0.9033 - val_loss: 0.2759 - val_accuracy: 0.8949
Epoch 10/100
3579/3579 - 0

**EVALUATE**


In [53]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 90us/sample - loss: 0.2438 - accuracy: 0.9085
